# **Spacecraft Example Model in SysMLv2**
# *INCOSE International Symposium July 2023*


# Definitions

In [ ]:
package Definitions {
    import ScalarValues::Real;
    
    item def Data {
        attribute value : Real;
    }
    item def Ping;
           
    port def CommunicationPort {
        out item data : Data;
        in item ping : Ping;
    }
    
    interface def CommunicationInterface {
        end groundPort : CommunicationPort;
        end spacePort : ~CommunicationPort;
        connect spacePort.data to groundPort.data;
        flow from groundPort.ping to spacePort.ping;
    }
}

# Components

In [ ]:
package Components {
    import Definitions::*;
    
    import SI::s;
    import ScalarValues::Real;
    
    item pingSignal : Ping;
    item shutDown : Ping;
    
    part def GroundStation {
        item startSignal;
        
        port commPort : ~CommunicationPort;
        
        action receiveData {
            first start;
            
            then while true action { // infinite loop
                accept data : Data via commPort;
                then action processData;
            }
            
            /* this would be the classic method
            then merge repeat;
            
            then accept data : Data via commPort;
            
            then action processData;
            
            then repeat;
            */
        }
        
        exhibit state behaviour {
            entry; then idle;
            
            state idle;
            
            transition first idle accept after 30 [s] then operation;
            transition first idle accept :> startSignal then operation;
            
            state operation {
                entry send pingSignal() via commPort;
                do receiveData;
            }
            
            transition first operation accept after 10 [s] then operation;
            transition first operation accept :> shutDown then idle;
        }
    }
    
    part def Spacecraft {
        item fullyCharged;
        item batteryLow;
        item transmissionDone;
        
        attribute data : Real = 1024;
        attribute battery : Real = 100;
        
        port commPort : CommunicationPort;
                
        action consumePower {
            first start;
            
            then loop action {
                assign battery := battery - 1;
            } until battery < 40; // battery < 40 --> terminates the loop
            
            then send batteryLow() to self;
            then done;
            
            /* this would be the classic method
            then merge repeat;            
            then assign battery := battery - 1;
            then decide;
                if battery >= 40 then repeat;
                else sendBatteryLow;
            
            action sendBatteryLow send batteryLow() to self;
            then done;
            */
                        
            action interrupt accept :> shutDown;
            then done; // Stop when 'shutDown' signal received
        }
        
        action rechargeBatteries {
            first start;
            
            then loop action {
                assign battery := battery + 1;
            } until battery >= 100; // battery >= 100 --> terminates the loop

            then send fullyCharged() to self;
            then done;
            
            /* this would be the classic method
            then merge repeat;            
            then decide;
                if battery < 100 then chargeBattery;
                else sendCharged;
            
            action chargeBattery assign battery := battery + 1;
            then repeat;
            
            action sendCharged send fullyCharged() to self;
            then done;
            */
        }
        
        action transmitData {
            first start;
            
            then fork forkNode;
                //then repeat;
                then consume;            
            
            action repeat loop action {
                send Data(data) via commPort;
                then assign data := data - 1;
            } until data <= 0; // data <= 0 --> terminates the loop
            
            then send shutDown() to self;
            then send shutDown() via commPort;
            then send transmissionDone() to self;
            then done; // no join -
            
            
            action consume :> consumePower;
            then done;
            
            /* this would be the classic method            
            merge repeat;            
            then decide;
                if data > 0 then sendData;
                else finished;
            
            action sendData {
                first start;
                then action send Data(data) via commPort;
                then assign data := data - 1;
                then done;
            }
            then repeat;
            
            action finished send shutDown() to self;
            then send shutDown() via commPort;
            then send transmissionDone() to self;
            then done;
            
            action consume :> consumePower;
            then done;
            */
        }
        
        exhibit state behaviour parallel { // state with multiple parallel regions
            state transmission {
                entry; then waitingPing;
                
                state waitingPing;
                
                transition first waitingPing accept :> pingSignal via commPort then transmitting;
                
                state transmitting {
                    do transmitData;
                }
                
                transition first transmitting accept :> transmissionDone then waitingPing;
                transition first transmitting accept :> batteryLow then waitingPing;
            }
            
            state charge {
                entry; then notRecharging;
                
                state notRecharging;
                
                transition first notRecharging accept when battery > 80 then recharging;
                
                state recharging {
                    do rechargeBatteries;
                }
                
                transition first recharging accept :> fullyCharged then notRecharging;
            }            
        }
        
    }
    
}

# System

In [ ]:
package System {

    import Components::*;
    
    part def Mission {        
        part groundStation : GroundStation;
        part spacecraft : Spacecraft;
        
        interface : Definitions::CommunicationInterface connect
            groundPort ::> groundStation::commPort to 
            spacePort  ::> spacecraft::commPort; 
    }

}